In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
)


In [9]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    serp_api_key = os.getenv("SERPAPI_KEY")

    if not serp_api_key:
        raise ValueError("SerpAPI key not found in environment variables")

    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": serp_api_key,
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("organic_results", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item.get("snippet", ""), "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    print(enriched_results)    

    return enriched_results


In [10]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )


    print(results)
    return results

In [11]:
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [12]:
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=azure_openai_chat_completion_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

In [13]:
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

In [14]:
await Console(
    team.run_stream(
        task="멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성",
        # task="최신 RAG 기술에 대한 동향 및 논문 리뷰",
    )
)

---------- user ----------
멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성


---------- Google_Search_Agent ----------
[FunctionCall(id='call_dx73sfKRUyQWkvcJFZdSBRcT', arguments='{"query":"멀티 에이전트 AI 시스템 구축 논문 최신 연구","num_results":2}', name='google_search')]
---------- Google_Search_Agent ----------
[FunctionExecutionResult(content="[{'title': 'AI 에이전트의 확산과 과제 | 인사이트리포트', 'link': 'https://www.samsungsds.com/kr/insights/challenges-of-ai-agents.html', 'snippet': '이 아티클은 인프라 현대화, 데이터 통합, 보안 및 규정 준수 문제를 중심으로 AI 에이전트 도입의 현실적 난관과 기회를 살펴봅니다.', 'body': 'AI 에이전트의 확산과 과제 | 인사이트리포트 | 삼성SDS 본문 바로 가기 인사이트 리포트 / 상세 기사 loading... 인공지능 AI 에이전트의 확산과 과제 2025-03-12 Maria Korolov 공유하기 인쇄하기 이 글은 IDG의 아티클을 전재하여 제공합니다. [원문보기] 복잡성과 보안이 주요 우려 요소입니다. 이에 더해 현대화된 인프라를 확보해야 한다는 과제도 있습니다. 작년 획기적인 기술로 이목을 집중시킨 에이전트형 AI가 올해에는 대규모 배치 단계에 진입할 전망입니다. 1월 KPMG의 대기업 고위 임원 100명을 대상으로 한 설문조사에 따르면, 12%의 기업이 이미 AI 에이전트를 배치하는 중입니다. 37%는 시범 단계에, 51%는 검토 단계에 있습니다. 그리고 10월에 발표된 가트너의 보고서에 따르면, 2033년에는 엔터프라이즈 소프트웨어의 33%가 에이전트형 AI를 포함할 전망입니다. 2024년의 수치는 1%입니다. 가트너는 일상적인 업무 결정의 15%가'}, {'title': '[논문]멀티 에이전트(

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성', type='TextMessage'), ToolCallRequestEvent(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=32), metadata={}, content=[FunctionCall(id='call_dx73sfKRUyQWkvcJFZdSBRcT', arguments='{"query":"멀티 에이전트 AI 시스템 구축 논문 최신 연구","num_results":2}', name='google_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Google_Search_Agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="[{'title': 'AI 에이전트의 확산과 과제 | 인사이트리포트', 'link': 'https://www.samsungsds.com/kr/insights/challenges-of-ai-agents.html', 'snippet': '이 아티클은 인프라 현대화, 데이터 통합, 보안 및 규정 준수 문제를 중심으로 AI 에이전트 도입의 현실적 난관과 기회를 살펴봅니다.', 'body': 'AI 에이전트의 확산과 과제 | 인사이트리포트 | 삼성SDS 본문 바로 가기 인사이트 리포트 / 상세 기사 loading... 인공지능 AI 에이전트의 확산과 과제 2025-03-12 Maria Korolov 공유하기 인쇄하기 이 글은 IDG의 아티클을 전재하여 제공합니다. [원문보기] 복잡성과 보안이 주요 우려 요소입니다. 이에 더해 현대화된 인프